<a href="https://colab.research.google.com/github/jp-ishimwe/Projects/blob/master/T5_ftransfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Import packages { run: "auto", vertical-output: true, display-mode: "both" }
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.base import BaseEstimator, TransformerMixin
import progressbar
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split


!pip install transformers -q

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau



SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Failed to initialize NVML: Driver/library version mismatch
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
#@title To proceed should be cuda { run: "auto", vertical-output: true }
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")
print(device)

cuda


#### Getting dataset

You can find the dataset [here](https://github.com/mahnazkoupaee/WikiHow-Dataset)

**Loading data**

In [3]:
# with open('/content/drive/My Drive/Colab Notebooks/wikihowAll.csv') as f:
#   wikihowAll = pd.read_csv(f)
# print(f'The size of data:{wikihowAll.shape}')

wikihowAll = pd.read_csv('wikihowAll.csv')

In [16]:
wikihowAll.head(2)

,headline,title,text
0,"\nKeep related supplies in the same area.,\nMa...",How to Be an Organized Artist1,"If you're a photographer, keep all the necess..."
1,\nCreate a sketch in the NeoPopRealist manner ...,How to Create a Neopoprealist Art Work,See the image for how this drawing develops s...


In [4]:
contraction = {"isn't": "is not", "aren't": "are not","can't": "cannot", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you ll":"you will", "you'll've": "you will have",
                           "you're": "you are", "you re": "you are" , "you've": "you have"}

###### Checking for NA

In [18]:
wikihowAll.isnull().sum()

headline     818
title          1
text        1071
dtype: int64

we have 818 articles without summaries

In [5]:
new_wikihowAll = wikihowAll[['text', 'headline']]
new_wikihowAll = new_wikihowAll.dropna(how='all')

In [20]:
new_wikihowAll.isnull().sum()

text        253
headline      0
dtype: int64

In [21]:
indices = np.where(new_wikihowAll['text'].isna())

In [22]:
new_wikihowAll.iloc[indices]

,text,headline
1492,NaN,Colour coordinate your looks (For all Events)\n\n
4328,NaN,\nRecognize the following symptoms:\n\nBloatin...
7190,NaN,"\nAt each practice, teach all of your squad di..."
7228,NaN,Understand that these directions assume the re...
7378,NaN,"Dribble with your left or right hand, whicheve..."
...,...,...
208605,NaN,\nStretch each arm as far across your chest wh...
210787,NaN,\nDownshift to a lower gear to get more power ...
212574,NaN,\nThe next thing you need to do is show Invisi...
213977,NaN,"\nIt is the same as the chorus, but with the s..."


In [6]:
new_wikihowAll = new_wikihowAll.dropna().reset_index(drop=True)
new_wikihowAll.isnull().sum()

text        0
headline    0
dtype: int64

In [24]:
new_wikihowAll.shape

(214294, 2)

In [15]:
# h = new_wikihowAll

# for x in ['text', 'headline']:
# 		h[x+'_tokenized'] = h[x].apply(lambda x:x.lower().split() );
# 		h[x+'_len'] = h[x+'_tokenized'].apply(lambda x: len(x))
# min(h['headline_len']), min(h['text_len']), len(h[h['headline_len']==1]), len(h[h['text_len']==0])

#####Preprocessing

In [7]:
class TextProccessing(BaseEstimator, TransformerMixin):
  def __init__(self, column, contraction):
    self.column = column
    self.contraction = contraction
  def fit(self, x, y=None):
    return self
  @staticmethod
  def preprocessing(line):
    
    line = line.lower() 
    line = re.sub(r"http\S+", " ",line)
    line = re.sub(r'[^\sa-zA-Z0-9.,!?]',' ',line)
    line = line.replace('-', ' ')
    line = line.strip(' ')
    line = re.sub(r'\s{2,}',' ', line)

    return line

  def transform(self, X):
    #X = X[self.column].replace(self.contraction)
    return X[self.column].replace(self.contraction).fillna('').apply(lambda x: self.preprocessing(x)) 

In [8]:
class Processor:

  def __init__(self, data, 
               contraction,
               max_length = None):
    
    self.max_length = max_length
    self.data = data
    self.contraction = contraction
    #self.textproc = textproc

  def cleaning(self, data):
    if not isinstance(data, pd.DataFrame):
      raise TypeError('Only Dataframes are allowed, but got data={}'.format(data))
    else:
      print('Please wait, we are cleaning...')
      titles = ['headline', 'text']
      data_dict = {}
      good = []

      for title in titles:
        proc = TextProccessing(title, self.contraction)
        res = proc.fit_transform(data)
        if title == 'headline':    
          for tt in res:
            row = ' '.join(tt.split('\n'))
            good.append(row.strip())
        data_dict[title] = res

      dd = pd.DataFrame(good)
      dd.columns = ['headline']
      ss = pd.DataFrame(data_dict)
      ss = ss.drop(columns='headline', axis=1)
      dataset = pd.concat([ss, dd], axis=1)
      
      return dataset

  @staticmethod
  def split(data):
    return len(data.split())

  def get_max_tokens(self, dataframe):

    dataframe['article_len'] = dataframe['text'].apply(self.split)
    dataframe['summary_len'] = dataframe['headline'].apply(self.split)
    
    max_len = {'max_article_tokens': max(dataframe['article_len']), 
                'min_article_tokens': min(dataframe['article_len']),
                'max_summary_tokens': max(dataframe['summary_len']),
                'min_summary_tokens': min(dataframe['summary_len'])}
    return max_len, dataframe

  def cleaned_data(self):
    dataframe = self.cleaning(self.data)
    
    if self.max_length is None:
      max_len, dataframe = self.get_max_tokens(dataframe)
    else:
      max_len = self.max_length

    min_length = max_len['min_article_tokens']
    idx_list = []
    for idx, paragraph in tqdm(enumerate(dataframe)):
        pragraph = len(paragraph.split())
        if pragraph <= min_length:
          idx_list.append(idx)
    final_data = dataframe.drop(idx_list)
    final_data = final_data.reset_index(drop=True)

    _, final_data = self.get_max_tokens(final_data)


    print('Done!')
    return final_data, max_len

In [9]:
maxlen = {'max_article_tokens': 512,
          'max_summary_tokens': 300,
          'min_article_tokens': 30,
          'min_summary_tokens': 0}

In [10]:
textproc = Processor(new_wikihowAll,contraction)#, maxlen)
dataset, max_len = textproc.cleaned_data()

Please wait, we are cleaning...



Done!


In [29]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214294 entries, 0 to 214293
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   text         214294 non-null  object
 1   headline     214294 non-null  object
 2   article_len  214294 non-null  int64 
 3   summary_len  214294 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 6.5+ MB


In [30]:
max_len

{'max_article_tokens': 12394,
 'max_summary_tokens': 4341,
 'min_article_tokens': 0,
 'min_summary_tokens': 1}

In [31]:
dataset['summary_len'].quantile([0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 0.9999])

0.5000      44.0000
0.7500      77.0000
0.9000     122.0000
0.9500     159.0000
0.9900     266.0000
0.9990     503.0000
0.9999    1024.9796
Name: summary_len, dtype: float64

In [32]:
dataset['article_len'].quantile([0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 0.9999])

0.5000     306.0000
0.7500     550.0000
0.9000    1049.0000
0.9500    1484.0000
0.9900    2364.0700
0.9990    3665.1210
0.9999    5201.5503
Name: article_len, dtype: float64

In [33]:
art = int(dataset['article_len'].quantile(0.9900))
sum = int(dataset['summary_len'].quantile(0.9900))

print(f'art len:{art}, sum len:{sum}')

art len:2364, sum len:266


#### Creating Custom Dataset

In [11]:
class CustomDataReader(Dataset):
  """
  Loading data to be used in pytorch pre-trained models
  
  """
  def __init__(self, dataframe, T5tokenizer, **kwargs):
    self.tokenizer = T5tokenizer
    self.dataframe = dataframe
    self.articles = dataframe.text
    self.summary  = dataframe.headline
    self.source_len = kwargs.get('max_article_tokens', 512)
    self.target_len = kwargs.get('max_summary_tokens', 150)
  
  def __len__(self):
    return len(self.articles)

  def __getitem__(self, idx):
    articles = self.articles[idx]
    articles = ' '.join(articles.split())

    summary = self.summary[idx]
    summary = ' '.join(summary.split())

    source_tokenized = self.tokenizer.batch_encode_plus([articles], max_length= self.source_len, pad_to_max_length=True, truncation=True,return_tensors='pt')
    target_tokenized = self.tokenizer.batch_encode_plus([summary], max_length= self.target_len, pad_to_max_length=True, truncation=True,return_tensors='pt')

    source_ids = source_tokenized['input_ids'].squeeze(0)
    source_mask = source_tokenized['attention_mask'].squeeze(0)

    summary_ids = target_tokenized['input_ids'].squeeze(0)
    summary_mask = target_tokenized['attention_mask'].squeeze(0)

    return source_ids, source_mask, summary_ids, summary_mask


######Training Loop

In [12]:
def training(model, dataset_loader, optimizer, iteration=0):

  model.train()
  avg_loss = 0


  print(f'Start training for iteration: {iteration}')
 
  for idx, data_val in enumerate(dataset_loader):

    input_ids, source_mask, summary_ids, summary_mask = data_val

    input_ids = input_ids.to(device, dtype = torch.long)
    source_mask = source_mask.to(device, dtype = torch.float)
    summary_ids = summary_ids.to(device, dtype = torch.long)

    target_labels = summary_ids[:, :-1].contiguous()
    langm_labels = summary_ids[:, 1:].clone().detach()
    langm_labels[summary_ids[:, 1:] == tokenizer.pad_token_id] = -100

    optimizer.zero_grad()
    
    output = model(input_ids=input_ids, attention_mask = source_mask, decoder_input_ids=target_labels, lm_labels=langm_labels, output_hidden_states=True)
    
    loss, prediction_scores, hidden_states = output[:3]

    loss.backward()
    optimizer.step()


    if idx % 100 == 0:
      print(f'The loss is:{loss:.3f} and of: {idx+1}/{(len(train_set)/int(len(input_ids)))} batches')
  
  return loss, prediction_scores, hidden_states


  
  #bar.finish()

In [13]:
#@title Train, Validation and Test splits { run: "auto" }
dataset.text = "summarize: " + dataset.text # for T5 conformity

columns = ["text","headline"]
df = dataset[columns]
X_train, X_test = train_test_split(df, test_size=0.1, random_state = 42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [37]:
X_train.head()

,text,headline
0,summarize: this calendar of events is availabl...,"consult the formula 1 race calendar., pick an ..."
1,summarize: once you ve finished your self anal...,"fix the problem., make a conscious effort to l..."
2,summarize: stir it thoroughly and then set it ...,add 1 teaspoon 4.9 ml of borax to 500 millilit...
3,summarize: recognizing that you need to establ...,"decide to set boundaries., define the boundary..."
4,summarize: communicate with your significant o...,talk with your significant other about your re...


In [ ]:
# train_size = 0.8
# train_dataset=df.sample(frac=train_size, random_state = SEED).reset_index(drop=True)
# val_dataset=df.drop(train_dataset.index).reset_index(drop=True)

In [38]:
d = ' '.join(X_train.iloc[0]['text'].split())
d

'summarize: this calendar of events is available on the official internet site of formula 1. the official formula 1 internet site also lists the scheduled times for the friday free practice sessions, the saturday free practice and race qualifying sessions, and the sunday race. there is some variation in race venues from year to year, so be sure to use the current race calendar. , the early season races will be more open to all competitors while in the later races, the competition will increasingly focus on those who still have a chance to be world champion. , the official formula 1 internet calendar will offer track maps for all events. these are not standard ovals, but rather complex courses with turns of varying difficulty and straights of varying lengths. choose a slower, twisty track or a wide, high speed track with long straights. , be aware that the typical weather expectations for each race venue will differ. for example, it may be hot at the turkish grand prix and rainy at the 

In [14]:
LEARNING_RATE = 1e-5
MODEL_NAME = 't5-base' # or T5-Large:770 params, T5-3B, T5-Base: 220 Params, T5-11B
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

model = model.to(device)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
#print(model)
X_train = X_train.iloc[:2000]
X_test = X_test.iloc[:600]

In [16]:
args = {'max_article_tokens': 512,
          'max_summary_tokens': 150}

train_set = CustomDataReader(X_train, tokenizer,**args)
val_set = CustomDataReader(X_test, tokenizer,**args)

In [17]:
BATCH_SIZE = 4
train_params = {
        'batch_size': BATCH_SIZE,
        'shuffle': True,
        'num_workers': 2
        }

val_params = {
    'batch_size': BATCH_SIZE,
    'shuffle': False,
    'num_workers': 2
    }

train_loader = DataLoader(train_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

In [18]:
N_EPOCHS = 5
best_loss = float('inf')
train_loss = []

for i in range(N_EPOCHS):
  loss, prediction_scores, hidden_states = training(model, train_loader, optimizer, iteration=i)
  train_loss.append(loss.item())

  if best_loss > loss:
    best_loss = loss
    torch.save(model.state_dict(), 'best_model.pth')
print(f'Best loss is: {best_loss:.3f}')
  #scheduler.step(prediction_scores)

Start training for iteration: 0
The loss is:9.027 and of: 1/500.0 batches
The loss is:4.217 and of: 101/500.0 batches
The loss is:3.788 and of: 201/500.0 batches
The loss is:3.575 and of: 301/500.0 batches
The loss is:3.113 and of: 401/500.0 batches
Start training for iteration: 1
The loss is:2.858 and of: 1/500.0 batches
The loss is:2.771 and of: 101/500.0 batches
The loss is:3.253 and of: 201/500.0 batches
The loss is:2.725 and of: 301/500.0 batches
The loss is:2.381 and of: 401/500.0 batches
Start training for iteration: 2
The loss is:3.025 and of: 1/500.0 batches
The loss is:2.551 and of: 101/500.0 batches
The loss is:2.729 and of: 201/500.0 batches
The loss is:2.783 and of: 301/500.0 batches
The loss is:2.878 and of: 401/500.0 batches
Start training for iteration: 3
The loss is:2.673 and of: 1/500.0 batches
The loss is:2.233 and of: 101/500.0 batches
The loss is:3.173 and of: 201/500.0 batches
The loss is:2.306 and of: 301/500.0 batches
The loss is:2.668 and of: 401/500.0 batches


In [37]:
def validation(model, dataset_loader, iteration=0):
  
  model.eval()

  results = {"summary": [], "article": []}

  with torch.no_grad():

    for idx, data_val in enumerate(dataset_loader):
      
      input_ids, source_mask, summary_ids, summary_mask = data_val 
      
      input_ids = input_ids.to(device, dtype = torch.long)
      source_mask = source_mask.to(device, dtype = torch.float)
      summary_ids = summary_ids.to(device, dtype = torch.long)

      generatation = model.generate(input_ids=input_ids, 
                                    decoder_attention_mask=source_mask, 
                                    min_length=30, 
                                    max_length=300,                          
                                    num_beams=3,
                                    repetition_penalty=2.5,
                                    eos_token_id = model.config.eos_token_id,               
                                    length_penalty=1.0, 
                                    early_stopping=True
                                    )

      for sc in summary_ids: # Taking whole document IDs and docode the summary
        #print(sc.size())
        sc_tokens_dec = tokenizer.decode(sc, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        results["Original_summary"].append(sc_tokens_dec)
        
      for gen in generatation:
        gen_tokens_dec = tokenizer.decode(gen, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        results["Generated_summary"].append(gen_tokens_dec)

  return results


In [29]:
results = validation(model, val_loader, iteration=0)

##### Examples and summaries 

In [41]:
X_test.iloc[0,:]['text']

'summarize: , this may be represented by a symbol representing a diode, an arrow pointing at a vertical line. ,, place the red probe on the positive side of the diode and the black probe on the negative side. if you get a reading of less than 1 but more than 0, the forward bias is good. , if the display reads ol overload , this indicates that reverse bias is good. , some multimeters will beep when the reading is less than 1. this doesn t necessarily indicate a good diode, because a shorted diode will also cause a beep. '

In [34]:
results['article'][0]

'insert the black probe into the common terminal and the red probe into the terminal marked for measuring ohms, volts or diode test., use the selector knob to choose the test diode function., turn the power off to the circuit., test forward bias., reverse the probes to test reverse bias., a reading of ol or 0 while testing forward bias, and 0 while testing reverse bias indicates that the diode is bad.'

In [40]:
results['summary'][0]

', place the red probe on the positive side of the diode and the black probe on negative side., read forward bias., check for ol overload., beep when the reading is less than 0., try to get a good display., use a shorted diometer., test for reverse bias., see if the display shows oh overload., check for ol overload., check for ol overload., check for ol overload., check for ole overload., find out how long it takes to measure the output voltage., turn off the display., set up an alarm., switch off the display., change the display., reset the display., adjust the display., select the display., choose the display., put the display., remove the display., repeat this step by step with your multimeter., stop the display., restart the display., start over again., enter the display., then click on the display., taping in the display., wait until you get a second timers that are not working., close the display., take the display., run the display., go backward-brightly position the display., l

In [42]:
X_test.iloc[1,:]['text']

'summarize: if you are in an altercation, reaching out to an authority figure can keep you out of trouble. schedule an appointment with your counselor and ask them for advice. you can explain your situation using fake names if that helps you feel more comfortable about opening up. you can reveal as much or as little as you choose. overall, your counselor will just want to make sure that you are safe., there is no shame in asking your peers for help if you are in trouble. call a friend that you trust. a friend can also have your back if you need them during the fight. come up with a signal word between the two of you so that your friend will know right away if you re in trouble., if you are anticipating a fight and hope to avoid it, walk around with a large group of people or with a trustworthy friend. if you outnumber the bully or if there is a large audience, he or she will feel less emboldened to threaten or intimidate you., trusted adults can give you helpful advice on how to handle

In [43]:
results['article'][1]

'talk to a counselor., ask a friend for assistance., travel in groups or with a friend., talk to parents, a coach, or a mentor.'

In [44]:
results['summary'][1]

"if you are in trouble, reach out to your counselor., ask for help., talk with friends., walk around., get help from trusted adults., avoid a fight altogether., get help from an adult who cares about your wellbeing and wants to help you., find therapists or other support groups., seek help from someone else., be prepared., try to avoid threatening situations., make sure that you are safe., don't feel ashamed., do not give up., stay calm., keep yourself"

In [45]:
X_test.iloc[10,:]['text']

'summarize: marshmallows are very sticky, so you need to start with a greased pan. spray down a 9 by 13 inch pan 23 centimeters by 33 centimeters with cooking spray. it can also help to spray down any utensils you re going to use. the mixture will stick to anything., mix the packets of gelatin with about half a cup 125 milliliters of water and the vanilla extract in the bowl of a stand mixer. just use a fork to mix them together. let the mixture sit to the side for a few minutes so the gelatin can bloom hydrate .try to get all the lumps out with the fork.\nyou can also use extracts like raspberry, lemon, orange, almond, or banana, depending on what you want to do with the marshmallows. , add 1 2 cup 125 milliliters of water to a small pan. pour in the sugar or the sugar plus the corn syrup, whichever you re using. bring it to a boil over medium to high heat, and let it boil until it reaches 250 degrees fahrenheit 121 degrees celsius or just under. continually wipe down the insides of t

In [46]:
results['article'][10]

'grease the pan., hydrate the gelatin., make the simple syrup., mix the gelatin in.'

In [48]:
results['summary'][10]

'a greased pan is best., add the gelatin and vanilla extract to melted butter., heat the mixture over medium-high heat., stir in the corn syrup., bring it to 125 degrees., raise the temperature of the mixture to 250 degrees., add the gelatin into the mixing bowl., let it cool down., add the gelatin to the pot., add the corn syrup., make sure you have enough water for your marshmallows., set aside., pour the gelatin into the saucepan., mix together the gelatin., boil the mixture., cook until the gelatin., put the gelatin on low heat., turn off the heat., remove the gelatin., cover with agar., use agar., place the gelatin., sprinkle some baking powder., bake them., roasting soda or sugar., simmer again., serve., cut out the gelatin., prepare the gelatin., allow the gelatin., beat well., then add the gelatin., combine all ingredients., reduce the gelatin., add the gelatin., add the gelatin., add the gelatin., add the gelatin., add the gelatin., add the gelatin., add the gelatin., add the 

######Performance

In [ ]:
def metric(labels, gen_target):
  print(f'Start evaluation')
  bar = progressbar.ProgressBar(maxval=len(dataset_loader))
  bar.start()






  bar.finish()


##### Freezing layers

In [ ]:
model.decoder.block[11].requires_grad=True
model.decoder.block[4].requires_grad=True

for param model.parameters():
    param.requires_grad = False

In [ ]:
class T5Model(nn.Module):

  def __init__(self, t5, )

In [ ]:
class DictionaryDataset:
  def __init__(self, dataset=None):
    self.w2index = defaultdict(int)
    self.index2w = defaultdict(lambda: 'SOS')
    self.symbols = ['SOS','EOS','PAD']
    self.wcount = 3

    for i, s in enumerate(self.symbols):
      self.index2w[i] = s
    for i, s in enumerate(self.symbols):
      self.w2index[s] = i
      
    if dataset:
      for i, line in tqdm(enumerate(dataset)):
        self.word2index(line)

  def word2index(self, sentence):
    words = sentence.split()
    for word in words:
      if word not in self.w2index.keys():
        self.w2index[word] = self.wcount
        self.index2w[self.wcount] = word
        self.wcount +=1
    
  def encode_sequence(self, dataset):
    for i, line in tqdm(enumerate(dataset)):
      self.word2index(line)

  def decode_idx(self, idx):
    dec_words = []
    for i in idx:
      dec_words.append(self.index2w[i])
    return dec_words

![LSTM](https://drive.google.com/open?id=19pYBGaKGx2wTHjDC3QLMkpr5HDqTPbb6)

![Seq2SeqWithAttention](https://drive.google.com/open?id=1QDLNN0XQ0TfeA-nyAGBhG3YgwAX0bPQU)

You should use the same number of layers, for encoder and decoder
shape for inputs is (batch size, sequence length), (batch size, 1) respectively.


In [ ]:
class Encoder(nn.Module):
  def __init__(self,input_size, embed_size, num_hidden, batch_size, num_layers=3, dropout = 0.1):
    super(Encoder, self).__init__()
    self.num_hidden = num_hidden
    self.num_layers = num_layers
    self.input_size = input_size
    self.batch_size = batch_size

    self.embed = nn.Embedding(input_size, embed_size)
    self.lstm = nn.LSTM(input_size=embed_size, hidden_size=num_hidden,num_layers = num_layers, dropout=(0 if num_layers ==2 else dropout))

  def init_hidden(self):
     # The axes semantics are (num_layers, minibatch_size, hidden_dim)
    return (torch.zeros(self.num_layers, self.batch_size, self.num_hidden), torch.zeros(self.num_layers, self.batch_size, self.num_hidden))
  def forward(self, input, hidden):
    # input (batch size, sequence length)
    embedding = self.embed(input)
    #print(embedding.shape)
    embed_out = embedding.transpose(0,1)
    print(embed_out.shape)
    lstm_out, hidden = self.lstm(embed_out, hidden)

    # The shape of lstm_out is (seq_len, batch_size, num_hiddens).
    # state contains the hidden state and the memory cell
    # of the last time step, the shape is (num_layers, batch_size, num_hiddens)
    #lstout is (seq_length, batch_size, num_hiddens)
    return lstm_out, hidden

In [ ]:
encoder = Encoder(input_size =10, embed_size=8, num_hidden=16, batch_size=4)
X = torch.zeros((4, 7),dtype=torch.long)
hidden = encoder.init_hidden()
output, hidden = encoder(X, hidden)

torch.Size([7, 4, 8])


In [ ]:
hidden[0].shape

torch.Size([3, 4, 16])

In [ ]:
output.shape

torch.Size([7, 4, 16])

In [ ]:
class Attention(nn.Module):
  def __init__(self, hidden_size, method='dot'):
    super(Attention, self).__init__()
    self.method = method
    self.hidden_size = hidden_size
  def dot_score(self, hidden, encoder_output):
    return torch.sum(hidden * encoder_output, dim=2)

  def forward(self, hidden, encoder_output):
    att_prod = self.dot_score(hidden, encoder_output)
    attn_weights = att_prod.t()
    return F.softmax(attn_weights, dim=1).unsqueeze(1)



In [ ]:
class DecoderWithAtt(nn.Module):
  def __init__(self, embed_size_in, num_hidden, output_size, att_method=None, num_layers = 3):
    super(DecoderWithAtt, self).__init__()
    self.num_layers = num_layers
    self.num_hidden = num_hidden
    self.output_size = output_size
    self.att_method = att_method

    self.emb = nn.Embedding(embed_size_in, num_hidden)
    self.lstm = nn.LSTM(input_size=num_hidden, hidden_size=num_hidden, num_layers = num_layers)
    self.con_ln = nn.Linear(num_hidden * 2, num_hidden)
    self.drop = nn.Dropout(p=0.2)
    self.fc  = nn.Linear(self.num_hidden, self.output_size)
    self.attention = Attention(att_method, num_hidden)

  def forward(self, prev_hidden, target_input, enconder_outputs):
    embed_out = self.emb(target_input).transpose(0,1)
    embed_out = self.drop(embed_out)
    print('embed out >>>', embed_out.shape)
    output, hidden = self.lstm(embed_out, prev_hidden) # shape (1, batch_size, hidden_size), 1 for one time step word 
    print('lstm out >>>', output.shape)
    print('hidden out >>>', hidden[0].shape)

    attent_weights = self.attention(output, enconder_outputs) # (att out shape: batch_size, 1, max_length), enc in shape: (max_length, batch_size, hidden_size)
    print('attent_weights out >>>', attent_weights.shape)
    print('enconder_outputs >>>', enconder_outputs.shape)

    context = torch.bmm(attent_weights, enconder_outputs.transpose(0,1))
    print('context out >>>', context.shape)
    cont_dec_out = torch.cat((context, output.transpose(0,1)), 1)
    print('to linear out >>>', cont_dec_out.shape)
    cont_dec_out = cont_dec_out.view(cont_dec_out.shape[0], -1)
    concat_out = self.con_ln(cont_dec_out)
    concat_out = torch.tanh(concat_out)

    out = self.fc(concat_out)
    out = F.softmax(out, dim=1)

    return out, hidden

In [ ]:
decoder = DecoderWithAtt(10,16,2)

In [ ]:
x = torch.zeros((4, 1),dtype=torch.long)
out, hidden = decoder(hidden, x, output)

embed out >>> torch.Size([1, 4, 16])
lstm out >>> torch.Size([1, 4, 16])
hidden out >>> torch.Size([3, 4, 16])
attent_weights out >>> torch.Size([4, 1, 7])
enconder_outputs >>> torch.Size([7, 4, 16])
context out >>> torch.Size([4, 1, 16])
to linear out >>> torch.Size([4, 2, 16])


In [ ]:
out.shape

torch.Size([4, 2])

In [ ]:
hidden[0].shape

torch.Size([3, 4, 16])